In [1]:
import fiona
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon, shape
from shapely.prepared import prep
import numpy as np

In [2]:
import shapely
print(shapely.__version__)

1.6.1


In [3]:
def pip(x,y,poly):
    n = len(poly)
    inside = False
    p1x,p1y = poly[0]
    for i in range(n+1):
        p2x,p2y = poly[i % n]
        if y > min(p1y,p2y):
            if y <= max(p1y,p2y):
                if x <= max(p1x,p2x):
                    if p1y != p2y:
                        xints = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
                    if p1x == p2x or x <= xints:
                        inside = not inside
        p1x,p1y = p2x,p2y
    return inside

In [4]:
def in_bbox(x,y,box):
    inside = False
    if (box[0] < x < box[2] and
                box[1] < y < box[3]):
        inside = not inside
    return inside

In [5]:
p1 = Point(-123.93, 45.0) # should be in aoi_poly
p2 = Point(-135.0, 45.0) # outside aoi_poly, inside aoi_hull and aoi_box
p3 = Point(-125.0, -45.0) # outside all

In [6]:
aoi_hull = fiona.open("/home/dmf/Recdev/data/TPL/data/closings_2hrdrives_wgs84/closings_2hrdrives_wgs84_hull.shp")
aoi_poly = fiona.open("/home/dmf/Recdev/data/TPL/data/closings_2hrdrives_wgs84/closings_2hrdrives_wgs84.shp")

In [7]:
print(len(aoi_hull))
print(len(aoi_poly))

1
5008


In [8]:
mpoly = MultiPolygon([shape(pol['geometry']) for pol in aoi_poly]).buffer(0)
poly = Polygon(shape(next(iter(aoi_hull))['geometry']))
prepared_mpoly = prep(mpoly)
prepared_poly = prep(poly)

In [9]:
type(prepared_mpoly)

shapely.prepared.PreparedGeometry

In [10]:
print(p1.within(mpoly))
print(p1.within(poly))
print(mpoly.contains(p1))
print(poly.contains(p1))

True
True
True
True


In [11]:
print(p2.within(mpoly))
print(p2.within(poly))

False
True


In [12]:
print(p3.within(mpoly))
print(p3.within(poly))

False
False


### shapely/GEOS `within()` is 10x faster than `pip()`

In [13]:
mpoly.bounds

(-159.57747058641803,
 17.766577350503802,
 -64.69942035868564,
 63.53866339451838)

In [14]:
poly.bounds

(-163.15036736719728,
 17.766577350503802,
 -64.69942035868564,
 63.54827841774561)

In [15]:
xs = np.random.uniform(low=-180.0, high=-30.0, size=1000)
ys = np.random.uniform(low=10.0, high=80.0, size=1000)

In [16]:
%%timeit -n50
# test on the single hull polygon
for x, y in zip(xs, ys):
    p = Point(x, y)
    p.within(poly)

34.1 ms ± 970 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)


In [17]:
%%timeit -n50
# test on the single hull polygon
for x, y in zip(xs, ys):
    pip(x,y, poly.exterior.coords)

388 ms ± 17 ms per loop (mean ± std. dev. of 7 runs, 50 loops each)


In [18]:
test = []
# test on the single hull polygon
for x, y in zip(xs, ys):
    p = Point(x, y)
    answer = p.within(poly)
    test.append(answer)

In [19]:
test2 = []
# test on the single hull polygon
for x, y in zip(xs, ys):
#     p = Point(x, y)
    answer = pip(x,y, poly.exterior.coords)
    test2.append(answer)

In [20]:
test == test2

True

In [21]:
np.sum(test)

346

### adding a simple bbox check first is a 30% improvement for the hull

In [22]:
aoibox = poly.bounds

In [23]:
%%timeit -n100
for x, y in zip(xs, ys):
    if in_bbox(x, y, aoibox):
        p = Point(x, y)
        p.within(poly)

22.6 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [24]:
%%timeit -n100
# test on the single hull polygon
for x, y in zip(xs, ys):
    p = Point(x, y)
    p.within(poly)

33.1 ms ± 182 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### testing on a collection of polygons, definitely prep geometry first, and still do bbox check first

In [25]:
# https://toblerity.org/shapely/manual.html#predicates-and-relationships
# a.contains(b) == b.within(a) always evaluates to True.

In [26]:
## too slow to wait for
# %%timeit -n10
# for x, y in zip(xs, ys):
#     p = Point(x, y)
#     p.within(mpoly)

In [27]:
## still too slow to wait for
# %%timeit -n10
# for x, y in zip(xs, ys):
#     if (aoibox[0] < x < aoibox[2] and aoibox[1] < y < aoibox[3]):
#         p = Point(x, y)
#         mpoly.contains(p)

In [28]:
%%timeit -n100
for x, y in zip(xs, ys):
    if in_bbox(x, y, aoibox):
        p = Point(x, y)
        prepared_mpoly.contains(p)

11.1 ms ± 608 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%%timeit -n100
for x, y in zip(xs, ys):
    p = Point(x, y)
    prepared_mpoly.contains(p)

21.2 ms ± 261 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### What about collecting a set of points first, testing all at once?
not as good if it means no bbox check before collecting points  
best yet if bbox check first, collecting only points inside  
but kind of a pain because of need to keep track of tweet metadata - username, date  
also, will require varying amounts of memory based on how many pts end up in collection

In [30]:
%%timeit -n100
mpts = MultiPoint([Point(x, y) for x,y in zip(xs, ys)])
prepared_mpoly.contains(mpts)

15.2 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
%%timeit -n100
mpts = MultiPoint([Point(x, y) for x,y in zip(xs, ys) if in_bbox(x, y, aoibox)])
prepared_mpoly.contains(mpts)

7.42 ms ± 83.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
### does last option work well in flow of twitter parser?
# load and prep a collection of polygons (AOI)
# loop through json lines to extract lon/lat from tweets:
    # check against bbox: 
        # collect points that pass, store in memory
        # but also need to store username, date
# check point collection against AOI:
    # write username, date, lon, lat%%timeit -n100

### Winning solution

In [33]:
%%timeit -n50
inside = False
for x, y in zip(xs, ys):
    if in_bbox(x, y, aoibox):
        p = Point(x, y)
        if prepared_mpoly.contains(p):
            inside = not inside

10.8 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)
